## Code to data clean step 1

In [ ]:
# import pandas as pd
# import re
# import os

# # Define file paths
# file_path = r"C:\Users\wenjie.soh.2022\Desktop\TextMining\project\emails.csv"
# output_file = r"C:\Users\wenjie.soh.2022\Desktop\TextMining\project\email_cleaned.csv"
# invalid_rows_log = r"C:\Users\wenjie.soh.2022\Desktop\TextMining\project\idk.csv"

# # Define batch size 
# batch_size = 5000

# # Function to validate email format
# def is_valid_email_format(email):
#     """Check if the email contains necessary fields."""
#     required_fields = ["Message-ID:", "From:", "To:", "Date:", "Subject:"]
#     return all(field in str(email) for field in required_fields)

# # Function to extract email components
# def extract_email_components(email, row_index):
#     """Extract metadata and content from an email."""
#     components = {
#         "Message-ID": None, "Date": None, "Time": None, "From": None, "To": None, "Subject": None,
#         "Mime-Version": None, "Content-Type": None, "Content-Transfer-Encoding": None,
#         "X-From": None, "X-To": None, "X-cc": None, "X-bcc": None, "X-Folder": None,
#         "X-Origin": None, "X-FileName": None, "Content": None
#     }

#     # Define regex patterns
#     patterns = {
#         "Message-ID": r"Message-ID:\s*(<.*?>)", "Date": r"Date:\s*(.+)", "From": r"From:\s*(.+)",
#         "To": r"To:\s*(.+)", "Subject": r"Subject:\s*(.*)", "Mime-Version": r"Mime-Version:\s*(.+)",
#         "Content-Type": r"Content-Type:\s*(.+)", "Content-Transfer-Encoding": r"Content-Transfer-Encoding:\s*(.+)",
#         "X-From": r"X-From:\s*(.+)", "X-To": r"X-To:\s*(.+)", "X-cc": r"X-cc:\s*(.*)",
#         "X-bcc": r"X-bcc:\s*(.*)", "X-Folder": r"X-Folder:\s*(.+)", "X-Origin": r"X-Origin:\s*(.+)",
#         "X-FileName": r"X-FileName:\s*(.+)"
#     }

#     for key, pattern in patterns.items():
#         match = re.search(pattern, email, re.MULTILINE)
#         if match:
#             components[key] = match.group(1).strip()

#     if components["Date"]:
#         date_parts = components["Date"].split(" ")
#         if len(date_parts) > 3:
#             components["Date"], components["Time"] = " ".join(date_parts[:3]), date_parts[3]

#     content_match = re.split(r"X-FileName:.*?\n", email, flags=re.S)
#     components["Content"] = content_match[1].strip() if len(content_match) > 1 else None

#     print(f"Processing row {row_index} ✅")
#     return components

# # Ensure the output file is not corrupted
# if os.path.exists(output_file):
#     try:
#         pd.read_csv(output_file)  # Try opening it
#     except Exception:
#         print("Corrupted file detected. Deleting and creating a new one.")
#         os.remove(output_file)

# # Load the CSV file
# df = pd.read_csv(file_path)

# # Prepare to store valid and invalid data
# valid_data = []
# invalid_data = []
# headers = ["file", "message"]

# # Process the dataframe row by row (skipping header implicitly via iteration)
# for index, row in df.iterrows():
#     file_name = row.get("file", "")
#     message = row.get("message", "")

#     # Remove leading/trailing spaces
#     message = str(message).strip() if message else ""

#     # Skip empty messages
#     if not message:
#         continue

#     # Separate valid and invalid rows
#     if is_valid_email_format(message):
#         valid_data.append([file_name, message])
#     else:
#         invalid_data.append([file_name, message])

#     # Process batch when reaching batch_size
#     if len(valid_data) >= batch_size:
#         print(f"\n🔹 Processing batch of {batch_size} valid emails...")

#         df_valid = pd.DataFrame(valid_data, columns=headers)
#         parsed_data = df_valid["message"].apply(lambda x, idx: extract_email_components(x, idx), args=(df_valid.index,)).apply(pd.Series)
#         df_cleaned = pd.concat([pd.Series(range(1, len(df_valid) + 1), name="No."), df_valid["file"], parsed_data], axis=1)

#         # Determine mode based on file existence
#         mode = "a" if os.path.exists(output_file) else "w"
        
#         # Append to output CSV file
#         df_cleaned.to_csv(output_file, mode=mode, header=(mode == "w"), index=False)

#         # Clear memory
#         valid_data = []

# # Save any remaining valid rows
# if valid_data:
#     df_valid = pd.DataFrame(valid_data, columns=headers)
#     parsed_data = df_valid["message"].apply(lambda x: extract_email_components(x, df_valid[df_valid["message"] == x].index[0])).apply(pd.Series)

#     df_cleaned = pd.concat([pd.Series(range(1, len(df_valid) + 1), name="No."), df_valid["file"], parsed_data], axis=1)

#     mode = "a" if os.path.exists(output_file) else "w"
    
#     df_cleaned.to_csv(output_file, mode=mode, header=(mode == "w"), index=False)

# # Save invalid rows
# if invalid_data:
#     df_invalid = pd.DataFrame(invalid_data, columns=headers)
#     df_invalid.to_csv(invalid_rows_log, mode="w", index=False)

# # Ensure at least one valid output exists
# if not os.path.exists(output_file) or os.path.getsize(output_file) == 0:
#     pd.DataFrame({"No Valid Data": ["No valid emails found"]}).to_csv(output_file, index=False)

# # print(f"Processing complete! Cleaned dataset saved to: {output_file}")
# # print(f"Invalid rows logged in: {invalid_rows_log}")

## Next is to apply the roles to the dataset


## Next is to trim the dataset columns and apply word count on the content column and further filter dataset

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)   # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Adjust the width to display full table
pd.set_option('display.max_colwidth', None)  # Adjust column width to avoid truncation


# Check file path
enron_emails_df=pd.read_csv('email_cleaned_with_roles.csv')

trimmed_enron_df=enron_emails_df.drop(columns=["X-FileName","X-Origin","Content-Transfer-Encoding","Content-Type","X-Folder","Mime-Version","X-To","X-From",'file','No.'])


filtered_df = trimmed_enron_df[~trimmed_enron_df["From"].isin(["enron.announcements@enron.com", "issuealert@scientech.com"])]

filtered_df.to_csv("trimmed_enron_emails.csv")

In [ ]:
df = pd.read_csv('trimmed_enron_emails.csv')

# test.to_csv('test.csv', index=False)
df = df.drop('Unnamed: 0', axis=1)
df.head()

In [ ]:
import pandas as pd
import re
import string
from tqdm.notebook import tqdm

def get_sentence_word_counts(text):
    # Split into sentences using . ! ? (followed by space or end of string)
    sentences = re.split('[.!?]+(?:\s+|$)', text)
    # Remove empty sentences and clean each one
    word_counts = []
    for sentence in sentences:
        if sentence.strip():  # Skip empty strings
            # Remove punctuation and newlines
            clean_sentence = sentence.translate(str.maketrans('', '', string.punctuation + '\n'))
            # Split into words and count
            words = clean_sentence.split()
            if words:  # Only include if there are words
                word_counts.append(len(words))
    return word_counts
tqdm.pandas()
# # Apply the function to the 'Content' column
df['Sentence_Word_Counts'] = df['Content'].progress_apply(get_sentence_word_counts)

def sum_word_counts(word_count_list):
    return sum(word_count_list)


# Apply the function to the 'Sentence_Word_Counts' column
df['Total_Sentence_Word_Count'] = df['Sentence_Word_Counts'].progress_apply(sum_word_counts)



In [ ]:
df_filtered = df[df['Total_Sentence_Word_Count'] >= 10]
df_filtered = df_filtered.drop('Sentence_Word_Counts', axis=1)
df_filtered = df_filtered.reset_index(drop=True)
df_filtered.to_csv('main_data.csv', index=False)

## Data cleaning ends here, next is further text pre-processing and maybe abit more data cleaning on the rows/columns